# Decision Trees

Your assignment is to write a CART decision tree from scratch, as we discussed in class.  Features should include:

  - Having a maximum depth parameter.

  - Ability to handle both categorical and continuous features

  - Creating predictions for rows after fitting

  - Ability to print the decision tree.  (see below for example)

Apply your implementation to two datasets:

1) The attached tennis dataset.  All features can be considered categorical.

2) The iris dataset: https://archive.ics.uci.edu/ml/datasets/iris (Links to an external site.).  All features are numeric.

Additionally, use the sklearn's implementation on the Adult dataset from Week 5's homework.  Be sure to tune at least 1 regularization hyperparameter (DecisionTreeClassifier for the available options).  Make a table comparing all the methods using Matthew's Correlation Coefficient.



# Decision Tree

In [2]:
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split

df = pd.read_csv('PlayTennis.csv')

In [3]:
def p(node, target):
  '''
  This function 
  '''
  counts = Counter(node[target])
  return np.sum([(freq/len(node))**2 for key, freq in counts.items()])

def gini_index(df, col, val, target):
  '''
  This function takes a col and splits it by the value an returns the gini index of this split
  '''
  if df[col].dtype == 'O' or df[col].dtype == 'bool':
    left = df[(df[col] == val)]
    right = df[(df[col] !=  val)]
  else:
    left = df[(df[col] <= val)]
    right = df[(df[col] > val)]
  p1 = p(left, target)
  p2 = p(right, target)
  # Get the predictions for each side. The prediction is just the category
  # (yes or no) that has the most observations 
  try:
    left_pred = left[[target]].value_counts().sort_values(ascending = False).index[0][0]
    right_pred = right[[target]].value_counts().sort_values(ascending = False).index[0][0]
  except Exception:
    left_pred = ''
    right_pred = ''
  return len(left)/len(df)*(1- p1) + len(right)*(1 - p2)/len(df), (left_pred, right_pred)

In [4]:
def get_best_gini(df, col, target):
  '''
  This functions loops through all the values of a column and
  return the column with the lowest gini index (the best one) for the split
  '''
  vals = df[col].unique()
  vals = sorted(vals)
  best_val = 0
  best_gini = 1
  best_preds = ''
  for val in vals:
    current_gini, current_preds  = gini_index(df, col, val, target)
    if current_gini < best_gini:
      best_val = val
      best_gini = current_gini
      best_preds = current_preds
      
  return best_val, best_gini, best_preds


In [5]:
def get_best_column(df, target):
  '''
  This function picks the best column (the one with the lowest gini index) in
  the data frame and returns the column and the correct split
  '''
  best_col = ''
  best_gini = 10
  split_val = 0
  best_preds = ''
  X = list(df.columns)
  
  X.remove(target)
  for col in X:
    current_val, current_gini, current_preds = get_best_gini(df, col, target)
    if current_gini < best_gini:
      best_col = col
      best_gini = current_gini
      split_val = current_val
      best_preds = current_preds
  return best_col, split_val, best_gini, best_preds


In [6]:
def tree(node, max_depth, target):
  '''
  This function takes a node, and a max_depth of the tree.
  It gets the best split and procedes until the max_depth is achieved.
  '''
  current_df = node[0]
  col, split_by, gini, preds = get_best_column(current_df, target)
  
  if node[1] > max_depth or node[3] == 0:
    full_tree[-1][3] = full_tree[-1][3] + '-END'
    return full_tree 

  #Split the tree by the best value and remove this column

  if current_df[col].dtype == 'O' or current_df[col].dtype == 'bool':
    left_path = current_df[(current_df[col] == split_by)]
    right_path = current_df[(current_df[col] !=  split_by)]
  else:
    left_path = current_df[(current_df[col] <= split_by)]
    right_path = current_df[(current_df[col] > split_by)]

  full_tree.append(node[1:])

  # Crete two new paths 
  left_node = [left_path, node[1] + 1, (col, ' ==/<= ' , split_by), gini, node[4] + 'L', preds[0]]
  right_node = [right_path, node[1] + 1, (col, '!=/> ', split_by), gini, node[4] + 'R', preds[1]]

  #Run the tree for these nodes again
  tree(left_node, max_depth, target)
  tree(right_node, max_depth, target)
  
  return full_tree

In [7]:
def print_tree(x):
  '''
  Print tree in a nicer way (?)
  '''
  if len(x) == 0:
    return
  if x[0][0] > 0:
    print(f'Node: {x[0][0]}', x[0][0]*'--','>', x[0][1][0], x[0])
  new_tree = x[1:]
  print_tree(new_tree)
  return

## Predict Function


In [8]:
path  = []
def walk_path(tree, row, path):
  '''
  This function returns the path that each row creates as it walks down the tree
  '''
  if len(tree) == 0:
    return []

  row_dict = dict(row._asdict())
  try:
    split_feature = tree[1][1][0]
  except IndexError:
    return [] 
  current_index = tree[0][0]
  if isinstance(row_dict[split_feature], float) or isinstance(row_dict[split_feature], int):
    if row_dict[split_feature] <= tree[1][1][2]:
      
      path.append('L')
    else:
      path.append('R')
  else:
    if row_dict[split_feature] == tree[1][1][2]:
      path.append('L')
    else:
      path.append('R')
  new_tree = [p for p in tree if p[3].startswith(''.join(path))]
  walk_path(new_tree, row, path)
  return path


def predict(tree, X_train):
  '''
  This function uses the walk_path function and applies it to the whole df 
  '''
  preds = []
  for row in X_train.itertuples():
    path = ''.join(walk_path(tree[1:], row, []))
    pred = [p[4] for p in tree if p[3] == path + "-END-END"]

    preds.append([row.Index, pred])
  return preds


In [9]:
def accuracy(y_preds, y_real):
  '''
  This function checks the accuracy of the tree
  '''
  y_p = [x[1][0] for x in y_preds]
  y_real = np.array(y_real.tolist())
  return np.sum(y_p == y_real)/len(y_p)

## Play Tennis data

To apply my function to a new data set I need a node 0 to initialize the tree. Each node will have the split variable and it's threshold value, the gini index of the previous split, the path, and the prediction of the y variable if you take the 'y'.

In [10]:
# The first node takes the current data, the depth, and the split history
node0 = [df, 0, '', 1, '', '']
max_depth = 5
full_tree = []
tenis_tree = tree(node0, max_depth, 'play')
print_tree(tenis_tree)

Node: 1 -- > outlook [1, ('outlook', ' ==/<= ', 'overcast'), 0.35714285714285715, 'L-END-END', 'yes']
Node: 1 -- > outlook [1, ('outlook', '!=/> ', 'overcast'), 0.35714285714285715, 'R', 'no']
Node: 2 ---- > humidity [2, ('humidity', ' ==/<= ', 'high'), 0.31999999999999984, 'RL', 'no']
Node: 3 ------ > outlook [3, ('outlook', ' ==/<= ', 'rainy'), 0.2, 'RLL-END-END', 'no']
Node: 3 ------ > outlook [3, ('outlook', '!=/> ', 'rainy'), 0.2, 'RLR-END-END', 'no']
Node: 2 ---- > humidity [2, ('humidity', '!=/> ', 'high'), 0.31999999999999984, 'RR', 'yes']
Node: 3 ------ > windy [3, ('windy', ' ==/<= ', False), 0.2, 'RRL-END-END', 'yes']
Node: 3 ------ > windy [3, ('windy', '!=/> ', False), 0.2, 'RRR-END-END', 'no']


## Iris Data Set

In [11]:
iris = pd.read_csv('iris.data', names = ['sepal-length'
                                          ,'sepal-widt'
                                          ,'petal-leng'
                                          ,'petal-width',
                                         'class'],
                   skiprows = 1)
iris.head()
# This transformation is neccesary for the way I did things. For some reason
# itertuples() does not read properly names with '-' in them.
iris.columns = iris.columns.str.replace('-','_')

In [12]:
node0 = [iris, 0, '', 1, '', '']
full_tree = []
iris_tree = tree(node0, max_depth = 8, target = 'class')
print_tree(iris_tree)

Node: 1 -- > petal_leng [1, ('petal_leng', ' ==/<= ', 1.9), 0.33557046979865773, 'L-END-END', 'Iris-setosa']
Node: 1 -- > petal_leng [1, ('petal_leng', '!=/> ', 1.9), 0.33557046979865773, 'R', 'Iris-versicolor']
Node: 2 ---- > petal_width [2, ('petal_width', ' ==/<= ', 1.7), 0.11030595813204513, 'RL', 'Iris-versicolor']
Node: 3 ------ > petal_leng [3, ('petal_leng', ' ==/<= ', 4.9), 0.0856481481481482, 'RLL-END-END', 'Iris-versicolor']
Node: 3 ------ > petal_leng [3, ('petal_leng', '!=/> ', 4.9), 0.0856481481481482, 'RLR', 'Iris-virginica']
Node: 4 -------- > petal_width [4, ('petal_width', ' ==/<= ', 1.5), 0.2222222222222222, 'RLRL-END-END', 'Iris-virginica']
Node: 4 -------- > petal_width [4, ('petal_width', '!=/> ', 1.5), 0.2222222222222222, 'RLRR-END-END', 'Iris-versicolor']
Node: 2 ---- > petal_width [2, ('petal_width', '!=/> ', 1.7), 0.11030595813204513, 'RR', 'Iris-virginica']
Node: 3 ------ > petal_leng [3, ('petal_leng', ' ==/<= ', 4.8), 0.02898550724637681, 'RRL-END-END', 'Ir

## Predictions

### On the tenis data

In [ ]:
cols =list(df.columns)
cols.remove('play') 
X_train = df.loc[:, cols]

for x in range(len(X_train)):
  print(df['play'].iloc[x], predict(tenis_tree, X_train)[x][1])

print(f"The accuracy of this tree is: {accuracy(predict(tenis_tree, X_train), df['play'])}")

no ['no']
no ['no']
yes ['yes']
yes ['no']
yes ['yes']
no ['no']
yes ['yes']
no ['no']
yes ['yes']
yes ['yes']
yes ['no']
yes ['yes']
yes ['yes']
no ['no']
The accuracy of this tree is: 0.8571428571428571


### On the iris data

In [13]:
iris = pd.read_csv('iris.data', names = ['sepal-length'
                                          ,'sepal-widt'
                                          ,'petal-leng'
                                          ,'petal-width',
                                         'class'],
                   skiprows = 1)

# This transformation is neccesary for the way I did things. For some reason
# itertuples() does not read properly names with '-' in them.
iris.columns = iris.columns.str.replace('-','_')

cols = list(iris.columns)
cols.remove('class')
X_iris = iris.loc[:,cols]

X_train, X_test, y_train, y_test = train_test_split(X_iris, iris['class'], test_size=0.33, random_state=42)

iris_train = pd.concat([X_train, y_train], axis = 1)
#iris_test = pd.concat([X_test, y_test], axis = 1)

node0 = [iris_train, 0, '', 1, '', '']
full_tree = []
iris_tree = tree(node0, max_depth = 8, target = 'class')
print_tree(iris_tree)


print(f"The accuracy of this tree is: {accuracy(predict(iris_tree, X_test), y_test)}")

Node: 1 -- > petal_leng [1, ('petal_leng', ' ==/<= ', 4.7), 0.3415945165945166, 'L-END-END', 'Iris-versicolor']
Node: 1 -- > petal_leng [1, ('petal_leng', '!=/> ', 4.7), 0.3415945165945166, 'R', 'Iris-virginica']
Node: 2 ---- > petal_width [2, ('petal_width', ' ==/<= ', 1.5), 0.03809523809523809, 'RL-END-END', 'Iris-virginica']
Node: 2 ---- > petal_width [2, ('petal_width', '!=/> ', 1.5), 0.03809523809523809, 'RR-END-END', 'Iris-virginica']
The accuracy of this tree is: 0.5


My tree is not that good in predicting when using 70 % of the data as training data. When I use the whole data frame it 'succesfully' over-fits the data and i get around 95% accuracy.

# Sklearn Tree

In [ ]:
from sklearn import preprocessing
from sklearn.metrics import matthews_corrcoef
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import matthews_corrcoef
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer

train = pd.read_csv('adult.data', names=['age',
              'workclass',
              'fnlwgt',
              'education',
              'education-num',
              'marital-status',
              'occupation',
              'relationship', 'race',
              'sex', 
              'capital-gain',
              'capital-loss',
              'hours-per',
              'native-country', 'y'], index_col = False).drop(['education'], axis = 1)

test = pd.read_csv('adult.test', names=['age',
              'workclass',
              'fnlwgt',
              'education',
              'education-num',
              'marital-status',
              'occupation',
              'relationship', 'race',
              'sex',
              'capital-gain',
              'capital-loss',
              'hours-per',
              'native-country', 'y'], index_col = False, skiprows = 1).\
              drop(['education'], axis = 1)

train['y'] = train['y'].apply(lambda x: x.replace('.', '').replace(' ', ''))
test['y'] = test['y'].apply(lambda x: x.replace('.', '').replace(' ', ''))

X_train = train.drop(['y'], axis = 1)
y_train = train['y']

X_test = test.drop(['y'], axis = 1)
y_test = test['y']

col_trans = make_column_transformer((OneHotEncoder(handle_unknown = 'ignore', 
                                                   drop = 'first'), 
                                    list(X_train.select_dtypes(include = 'O').columns)),
                                    (StandardScaler(), 
                                     list(X_train.select_dtypes(exclude = 'O').columns)),
                                    remainder = 'passthrough'
                                     )

tree_pipe = Pipeline(steps = [
                         ('preprocess', col_trans), 
                         ('model', DecisionTreeClassifier())
                         ])


parameters = {'model__max_depth': [i + 1  for i in range(50)]}

clf_pipe = GridSearchCV(tree_pipe, parameters, cv = 5,
                        scoring = make_scorer(matthews_corrcoef))

clf_pipe.fit(X_train, y_train)
y_preds = clf_pipe.predict(X_test)

accuracy_score(y_test, y_preds)

In [ ]:
methods_table = pd.DataFrame({"Max Depth": [i + 1 for i in range(15)], 
              "Matthews Correlation Coefficient": clf_pipe.cv_results_['mean_test_score'][:15]})


def color_best(val):
  if val == methods_table[['Matthews Correlation Coefficient']].max()[0]:
    color = 'green'
  else:
    color = 'white'
  return 'color: %s' % color

methods_table.style.applymap(color_best)

The grid search found that the optimal tree depth is 10, because it hs the highest matthews correlation coefficient : 0.58.

